In [ ]:
import geopandas as gpd
import psycopg2
from shapely.wkt import loads as wkt_loads
from keys import pg_user, pg_pass, pg_host, pg_port, pg_db

output_path = 'data/points_joined_tracts.csv'

In [ ]:
connection = psycopg2.connect(database=pg_db,
                              user=pg_user,
                              password=pg_pass,
                              host=pg_host,
                              port=pg_port)
cursor = connection.cursor()

In [ ]:
# verify that the SRIDs match
cursor.execute("""SELECT
                      Find_SRID('public', 'tracts', 'geom') as tracts_srid,
                      Find_SRID('public', 'points', 'geom') as points_srid""")
cursor.fetchall()

In [ ]:
# joining how many rows to how many rows?
cursor.execute("SELECT count(*) AS row_count FROM tracts")
count_tracts_table = cursor.fetchall()[0][0]
cursor.execute("SELECT count(*) AS row_count FROM points")
count_points_table = cursor.fetchall()[0][0]
count_tracts_table, count_points_table

## Do the spatial join

In [ ]:
%%time
# drop the table if it already exists and recreate it
# select distinct to keep only one point/tract pair even if the point lies on the border of multiple tracts
query = """
        DROP TABLE IF EXISTS points_tracts;
        CREATE TABLE points_tracts AS
        SELECT DISTINCT ON (point_id)
            points.id AS point_id,
            points.date AS date,
            points.region AS region,
            points.bedrooms AS bedrooms,
            points.rent AS rent,
            points.sqft AS sqft,
            points.geom AS geometry,
            tracts.geoid AS tract_geoid
        FROM points LEFT JOIN tracts
        ON ST_Intersects(points.geom, tracts.geom)
        """

cursor.execute(query)
connection.commit()

In [ ]:
%%time
# vacuum and analyze the database to optimize it after building indices and inserting rows
old_isolation_level = connection.isolation_level
connection.set_isolation_level(0)
cursor.execute("VACUUM ANALYZE")
connection.commit()
connection.set_isolation_level(old_isolation_level)

## Retrieve the data

In [ ]:
# some points still lack tract geoid, as they are offshore or just across canada/mexico border
cursor.execute("SELECT count(*) AS row_count, count(tract_geoid) AS geoid_count FROM points_tracts")
print(count_points_table)
cursor.fetchall()

In [ ]:
%%time
# select the results from the new table, ignoring any points with null tract_geoid
# because they are offshore or just across canada/mexico border
cursor.execute("""SELECT date, region, bedrooms, rent, sqft, ST_AsText(geometry), tract_geoid
                  FROM points_tracts
                  WHERE tract_geoid IS NOT NULL""")
rows = cursor.fetchall()

In [ ]:
cursor.close()
connection.close()

In [ ]:
%%time
# turn the selected rows into a geodataframe
gdf = gpd.GeoDataFrame(rows, columns=['date', 'region', 'bedrooms', 'rent', 'sqft', 'geometry', 'tract_geoid'])
gdf['geometry'] = gdf['geometry'].map(lambda x: wkt_loads(x))
gdf['lat'] = gdf['geometry'].map(lambda pt: pt.y)
gdf['lng'] = gdf['geometry'].map(lambda pt: pt.x)
gdf = gdf.drop('geometry', axis=1)

In [ ]:
print(len(gdf))
gdf.head()

In [ ]:
%%time
gdf.to_csv(output_path, encoding='utf-8', index=False)